# Upload a training file

In [3]:
import os
from dotenv import load_dotenv

from openai import OpenAI

load_dotenv()
client = OpenAI()

client.files.create(
  file=open("data/train.jsonl", "rb"),
  purpose="fine-tune"
)

FileObject(id='file-c3mUDtFLArm6II0HkJnwdkZd', bytes=5524, created_at=1720005473, filename='train.jsonl', object='file', purpose='fine-tune', status='processed', status_details=None)

### Create a fine-tuned model

In [7]:
client.fine_tuning.jobs.create(
  training_file="file-LzuXDbP1sp303stUPt3ddxvE", 
  model="gpt-3.5-turbo"
)

FineTuningJob(id='ftjob-e8uXubgFWvcpNxnmQz4SmfXC', created_at=1720003762, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(n_epochs='auto', batch_size='auto', learning_rate_multiplier='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-46nm54SkZfsldjgAVIMz9XfX', result_files=[], seed=1190866367, status='validating_files', trained_tokens=None, training_file='file-LzuXDbP1sp303stUPt3ddxvE', validation_file=None, estimated_finish=None, integrations=[], user_provided_suffix=None)

In [9]:
# List 10 fine-tuning jobs
client.fine_tuning.jobs.list(limit=10)

# Retrieve the state of a fine-tune
client.fine_tuning.jobs.retrieve("ftjob-e8uXubgFWvcpNxnmQz4SmfXC")

# List up to 10 events from a fine-tuning job
client.fine_tuning.jobs.list_events(fine_tuning_job_id="ftjob-e8uXubgFWvcpNxnmQz4SmfXC", limit=10)


SyncCursorPage[FineTuningJobEvent](data=[FineTuningJobEvent(id='ftevent-hoYyfS1dlaRy1kk0KzxyH1Lg', created_at=1720003960, level='info', message='Step 41/99: training loss=0.33', object='fine_tuning.job.event', data={'step': 41, 'train_loss': 0.32764625549316406, 'total_steps': 99, 'train_mean_token_accuracy': 0.8684210777282715}, type='metrics'), FineTuningJobEvent(id='ftevent-YMsMNJ9I3ju2k2wbAyvqCrJG', created_at=1720003958, level='info', message='Step 40/99: training loss=0.18', object='fine_tuning.job.event', data={'step': 40, 'train_loss': 0.17906267940998077, 'total_steps': 99, 'train_mean_token_accuracy': 0.9743589758872986}, type='metrics'), FineTuningJobEvent(id='ftevent-W9NHf8N8x2GiBIrlWBy1rjNv', created_at=1720003956, level='info', message='Step 39/99: training loss=0.00', object='fine_tuning.job.event', data={'step': 39, 'train_loss': 0.004266641102731228, 'total_steps': 99, 'train_mean_token_accuracy': 1.0}, type='metrics'), FineTuningJobEvent(id='ftevent-EzQLMV6dp8T4Hh3Uf0

### Use a fine-tuned model

In [4]:
completion = client.chat.completions.create(
  model="ft:gpt-3.5-turbo-0125:personal::9gruvYXE",
  messages=[
    {"role": "system", "content": "Given a batch of Chinese product models, please provide the product model , provide the following fields in English in a JSON dict, where applicable, Some of it can be inferred based on common sense: \"Size\" (inch), \"Color\", \"notes\"."},
    {"role": "user", "content": "北欧实木床头柜小户型卧室时尚中古床边柜家用创意带抽屉储物柜 松木：45*35*50cm"}
  ]
)
print(completion.choices[0].message)

ChatCompletionMessage(content='{"Size": "17.7\\"L x 13.8\\"W x 19.7\\"H", "Color": "Natural", "notes": "null"}', role='assistant', function_call=None, tool_calls=None)


In [15]:
print(completion.choices[0].message.content)

{"Size": "17.7\"L x 13.8\"W x 19.7\"H", "Color": "Natural", "notes": "Without Handles"}


In [9]:
import json
result_file = "completion_result.json"
content_json_str = completion.choices[0].message.content
content_dict = json.loads(content_json_str)  # 解析 JSON 字符串为字典

print(content_dict)

with open(result_file, "w", encoding="utf-8") as f:
    json.dump(content_dict, f, ensure_ascii=False, indent=4)

print(f"Result saved to {result_file}")

{'Size': '17.7"L x 13.8"W x 19.7"H', 'Color': 'Natural', 'notes': 'null'}
Result saved to completion_result.json
